## Victor Brivet 
### ID : 93734
### DATA INTEGRATION WITH APIS AND ETL PROCESSES

Imports :

In [3]:
import requests
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import time

### Alpha-Vantage API : 

#### EXTRACTION

We define the API key and specify the currency pair (EUR/USD) for which we will retrieve foreign exchange data.

In [4]:
# API ID
API_KEY = "J2OZYB9ZL1RBF9ZP"  
FROM_SYMBOL = "EUR"       # Devise de départ
TO_SYMBOL = "USD"         # Devise cible


We construct the request URL with the necessary parameters and send a GET request to retrieve the monthly foreign exchange data.
If the request fails, an exception is raised to indicate the error. We use a scalable solution in the ID so it's easy to change the currencies.

In [5]:
# URL
url = f"https://www.alphavantage.co/query?function=FX_MONTHLY&from_symbol={FROM_SYMBOL}&to_symbol={TO_SYMBOL}&apikey={API_KEY}"

# GET Request
response = requests.get(url)
if response.status_code != 200:
    raise Exception(f"Erreur API: {response.status_code} - {response.text}")

data = response.json()

We extract the "Time Series FX (Monthly)" section from the API response, which contains the monthly exchange rate data.

In [6]:
fx_data = data["Time Series FX (Monthly)"]

#### TRANSFORMATION

##### DataFrame Creation and Cleaning
We convert the extracted JSON data into a Pandas DataFrame, ensuring proper formatting:
- Convert the index (dates) to `datetime` format.
- Rename columns for clarity (`Open`, `High`, `Low`, `Close`).
- Convert values to `float` for numerical analysis.
- Sort the DataFrame by date in ascending order.

##### Data Quality Checks
We perform key quality checks:
- **Missing Values:** Identify if any data points are missing.
- **Outlier Detection:** Check for abnormal exchange rates (e.g., values outside the expected range of 0.5 to 2.0).


In [7]:
## Convert
df = pd.DataFrame.from_dict(fx_data, orient="index")
df.index = pd.to_datetime(df.index)  # Convertir les dates
df.columns = ["Open", "High", "Low", "Close"]
df = df.astype(float)  # Convertir les valeurs en float
df = df.sort_index()  # Trier les dates dans l'ordre croissant

## Missing Values
missing_values = df.isnull().sum()
print("Missing Values check :\n", missing_values)


## Outlier Detection
for col in ["Open", "High", "Low", "Close"]:
    if (df[col] < 0.5).any() or (df[col] > 2.0).any():
        print(f"Warning : Outliers {col} !")




Missing Values check :
 Open     0
High     0
Low      0
Close    0
dtype: int64


To enhance our analysis, we compute several key metrics based on the FX monthly data:

1. **Monthly Change (Open to Close) %**  
   - Measures the percentage change from the opening to the closing price.  
   - Formula: `((Close - Open) / Open) * 100`

2. **Range %**  
   - Represents the percentage difference between the highest and lowest price within the month.  
   - Formula: `((High - Low) / Low) * 100`

3. **Closing Momentum**  
   - Indicates whether the closing price is closer to the highest or lowest price of the month.  
   - Formula: `(Close - Low) / (High - Low)`

4. **Volatility Index**  
   - Measures the relative volatility of the exchange rate based on the high-low range.  
   - Formula: `((High - Low) / Close) * 100`

5. **Momentum Score**  
   - Evaluates the strength of the price movement relative to its range.  
   - Formula: `(Close - Open) / (High - Low)`

These calculated indicators help to better understand price movements, market volatility, and potential trading opportunities.

In [8]:
## Monthly Change (Open to Close) %
df["Monthly Change (Open to Close) %"] = ((df["Close"] - df["Open"]) / df["Open"]) * 100

## Range %
df["Range %"] = ((df["High"] - df["Low"]) / df["Low"]) * 100

## Closing Momentum
df["Closing Momentum"] = (df["Close"] - df["Low"]) / (df["High"] - df["Low"])

## Volatility Index
df["Volatility Index"] = ((df["High"] - df["Low"]) / df["Close"]) * 100

## Momentum Score
df["Momentum Score"] = (df["Close"] - df["Open"]) / (df["High"] - df["Low"])

Before exporting the data to a database, we rename certain columns to avoid conflicts with SQL reserved keywords and improve readability:

- **"Open" → "open_price"**  
- **"High" → "high_price"**  
- **"Low" → "low_price"**  
- **"Close" → "close_price"**  
- **"Range" → "range_value"** (avoiding conflicts with the SQL `RANGE` function)

After renaming, we print the column names to verify the changes.

In [9]:
## Renaming
df.rename(columns={
    "Open": "open_price",
    "High": "high_price",
    "Low": "low_price",
    "Close": "close_price",
    "Range": "range_value"
}, inplace=True)

## Check new names
print(df.columns)

Index(['open_price', 'high_price', 'low_price', 'close_price',
       'Monthly Change (Open to Close) %', 'Range %', 'Closing Momentum',
       'Volatility Index', 'Momentum Score'],
      dtype='object')


##### Preparing Data for SQL Export
Before exporting to a SQL database, we ensure the DataFrame is correctly formatted:

1. **Reset Index & Convert Date Column**  
   - Since the date was the index, we reset it as a regular column (`Date`).  
   - Renamed `"index"` to `"Date"` for clarity.  
   - Converted `"Date"` to `datetime64[ns]` for SQL compatibility.

2. **Convert Numeric Columns & Round Values**  
   - Ensures all numerical values are stored as `float`.  
   - Rounds values to 4 decimal places to maintain precision while reducing storage size.

3. **Final Data Type Verification**  
   - Prints the data types to confirm correct formatting before exporting to the database.

In [10]:
## Reset Index & Convert Date Column
df.reset_index(inplace=True)
df.rename(columns={"index": "Date"}, inplace=True)

## Convert Numeric Columns & Round Values
df["Date"] = df["Date"].astype("datetime64[ns]")  # Format SQL-compatible
df.iloc[:, 1:] = df.iloc[:, 1:].astype(float).round(4)  # Arrondir toutes les valeurs numériques

## Final Data Type Verification
print(df.dtypes)


Date                                datetime64[ns]
open_price                                 float64
high_price                                 float64
low_price                                  float64
close_price                                float64
Monthly Change (Open to Close) %           float64
Range %                                    float64
Closing Momentum                           float64
Volatility Index                           float64
Momentum Score                             float64
dtype: object


##### Standardizing Column Names for SQL Compatibility
To ensure smooth integration with SQL databases, we apply the following transformations to column names:

1. **Replace Spaces with Underscores (`_`)**  
   - Prevents issues when referencing columns in SQL queries.

2. **Convert Column Names to Lowercase**  
   - Ensures consistency and avoids case sensitivity issues in certain SQL environments.

3. **Final Verification**  
   - Prints the modified column names to confirm the changes.


In [11]:
# Replace Spaces with Underscores (`_`)
df.columns = df.columns.str.replace(" ", "_").str.lower()

# Convert Column Names to Lowercas
print(df.columns)



Index(['date', 'open_price', 'high_price', 'low_price', 'close_price',
       'monthly_change_(open_to_close)_%', 'range_%', 'closing_momentum',
       'volatility_index', 'momentum_score'],
      dtype='object')


#### LOAD

The cleaned and processed DataFrame is exported to a CSV file for further use or database integration.

1. **File Name:** `"API.csv"`
2. **Separator:** `","` (Comma-separated values, standard format)
3. **Encoding:** `"UTF-8"` to ensure compatibility with different systems.
4. **Index:** `False` (Prevents Pandas from adding an extra index column).

A success message confirms the file has been saved correctly.

In [12]:
csv_path = "Alpha_Vantage_Forex.csv"
df.to_csv(csv_path, sep=",", encoding="utf-8", index=False)

print(f"Export successful {csv_path}")

Export successful Alpha_Vantage_Forex.csv


### YAHOO FINANCE

#### EXTRACTION

We don't need an API key it's native.

Function: Download Stock Price Data
This function fetches historical stock price data using the `yfinance` library.

**Parameters:**
- **`ticker`** *(str)* → The stock symbol (e.g., "MSFT" for Microsoft).
- **`start`** *(str)* → Start date in `"YYYY-MM-DD"` format.
- **`end`** *(str)* → End date in `"YYYY-MM-DD"` format.


In [13]:
def get_stock_data(ticker, start, end):
    stock_data = yf.download(ticker, start=start, end=end)
    return stock_data


##### User Input: Stock Tickers and Date Range

**This section allows the user to specify:**
1. **The list of stock tickers** to analyze.
2. **The start and end date** for fetching historical data.

**Stock Selection**
- The user can either **manually enter tickers** or use a predefined list.
- Here, a predefined list of **CAC 40 stock symbols** is used.

**Date Range Selection**
- Users can input a **custom start and end date**.
- **Example range:** `"2010-01-01"` to `"2025-01-01"`.

**Validation**
- Ensures that the start date is **earlier** than the end date.
- If not, it raises an error:  
  ```python
  ValueError("La date de début doit être antérieure à la date de fin.")


In [14]:
# Prompt the user to enter the stock tickers to analyze
# tickers = input("Enter the stock tickers separated by spaces (e.g., MC.PA AIR.PA OR.PA SAN.PA BNP.PA): ").split()
tickers = [
    "AC.PA", "AI.PA", "AIR.PA", "MT.AS", "CS.PA", "BNP.PA", "EN.PA", "CAP.PA",
    "CA.PA", "ACA.PA", "BN.PA", "DSY.PA", "EDEN.PA", "ENGI.PA", "EL.PA", "ERF.PA",
    "RMS.PA", "KER.PA", "OR.PA", "LR.PA", "MC.PA", "ML.PA", "ORA.PA", "RI.PA",
    "PUB.PA", "RNO.PA", "SAF.PA", "SGO.PA", "SAN.PA", "SU.PA", "GLE.PA", "STLAP.PA",
    "STMPA.PA", "TEP.PA", "HO.PA", "TTE.PA", "URW.PA", "VIE.PA", "DG.PA", "VIV.PA"
]

# Choose the time period
# start_date = input("Enter the start date (YYYY-MM-DD, e.g., 2010-01-01) : ").strip()
# end_date = input("Enter the end date (YYYY-MM-DD, e.g., 2024-01-01) : ").strip()
start_date = '2010-01-01'
end_date = '2025-01-01'

# Validate the entered dates
if start_date >= end_date:
    raise ValueError("The start date must be earlier than the end date.")


In this section, we retrieve historical stock price data for the selected tickers and format it for further analysis.

1. **Download Historical Data**  
   - Uses the `get_stock_data()` function to fetch stock prices from Yahoo Finance for the specified tickers and date range.

2. **Reshape Data for Tableau Compatibility**  
   - The dataset is **stacked** to transform it into a **long format**.
   - **`reset_index()`** ensures a clean structure for visualization.
   - **Column names are dynamically adjusted** to prevent errors.

In [15]:
# Download historical stock price data
stock_prices = get_stock_data(tickers, start_date, end_date)

# Reshape the data for Tableau compatibility
stock_prices = stock_prices.stack(future_stack=True).reset_index()
stock_prices.columns = ["Date", "Ticker"] + list(stock_prices.columns[2:])  # Dynamically rename to avoid errors


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  40 of 40 completed


#### TRANSFORMATION

This section computes essential financial indicators to analyze stock performance and risk.

1. **Daily Return (`Daily Return`)**  
   - Measures the percentage change in stock price from one day to the next.  
   - Formula: `pct_change(fill_method=None)`

2. **Cumulative Return (`Cumulative Return`)**  
   - Tracks the compounded return of an asset over time.  
   - Formula: `(1 + Daily Return).cumprod()`

3. **Simple Moving Averages (`SMA_50`, `SMA_200`)**  
   - Computes the **50-day and 200-day** moving averages to analyze trends.  
   - Helps identify **short-term and long-term** trends.

4. **Risk-Free Rate Conversion**  
   - Assumes an **annual risk-free rate of 2%**.  
   - Converts it into a **daily rate**: `(1 + rate)^(1/252) - 1`

5. **Sharpe Ratio Calculation (`Sharpe Ratio`)**  
   - Measures the **risk-adjusted return** of an asset.  
   - Formula:  
     \[
     \frac{\text{Mean(Daily Return)} - \text{Risk-Free Rate}}{\text{Standard Deviation(Daily Return)}}
     \]
   - A higher Sharpe Ratio indicates **better risk-adjusted returns**.

6. **Handling Missing Values (`NaN`)**  
   - Uses **backfill (`bfill()`)** to fill missing values based on the next available data.  
   - Replaces remaining `NaN` values with `0` as a final safeguard.

These indicators help evaluate stock performance, momentum, and volatility for **data-driven investment decisions**.


In [16]:
# Calculate daily return
stock_prices["Daily Return"] = stock_prices.groupby("Ticker")["Close"].pct_change(fill_method=None)

# Calculate cumulative return
stock_prices["Cumulative Return"] = (1 + stock_prices["Daily Return"]).groupby(stock_prices["Ticker"]).cumprod()

# Compute 50-day and 200-day simple moving averages (SMA)
stock_prices["SMA_50"] = stock_prices.groupby("Ticker")["Close"].rolling(window=50).mean().reset_index(level=0, drop=True)
stock_prices["SMA_200"] = stock_prices.groupby("Ticker")["Close"].rolling(window=200).mean().reset_index(level=0, drop=True)

# Define the annual risk-free rate (e.g., 2%)
risk_free_rate_annual = 0.02  # 2%

# Convert to daily risk-free rate
risk_free_rate_daily = (1 + risk_free_rate_annual) ** (1/252) - 1  

# Compute the adjusted Sharpe Ratio
stock_prices["Sharpe Ratio"] = stock_prices.groupby("Ticker")["Daily Return"].transform(
    lambda x: (x.mean() - risk_free_rate_daily) / (x.std() + 1e-8)
)

# Handle missing values
stock_prices.bfill(inplace=True)  # Backfill to prevent data loss
stock_prices.fillna(0, inplace=True)  # Final safeguard against NaN values


##### Fetching and Merging Sector & Industry Data
This section retrieves **sector** and **industry** information for each stock ticker using the `yfinance` API and merges it with the existing stock price dataset.

 **Steps:**
1. **Create a dictionary to store sector & industry data.**
   - Loops through each ticker.
   - Fetches `sector` and `industry` information from Yahoo Finance.
   - Stores the data in a dictionary (`sector_industry_dict`).
   - If no data is available, it defaults to `"Unknown"`.

2. **Handle exceptions gracefully.**
   - If an error occurs while fetching the data, it prints the issue but continues processing.

3. **Convert the dictionary into a DataFrame.**
   - The dictionary is transformed into a **Pandas DataFrame**.
   - Column names are set to `"Ticker"`, `"Sector"`, and `"Industry"`.

4. **Merge with the stock price dataset.**
   - Uses a **left join** to ensure that stock price data remains intact while adding sector & industry information.

This allows for **sector-based** and **industry-based** analysis when evaluating stock performance.


In [17]:
# Dictionary to store sector and industry data
sector_industry_dict = {}

for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        sector = info.get("sector", "Unknown")   # Retrieve sector
        industry = info.get("industry", "Unknown")  # Retrieve industry
        sector_industry_dict[ticker] = {"Sector": sector, "Industry": industry}
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

# Convert the dictionary into a DataFrame
sector_industry_df = pd.DataFrame.from_dict(sector_industry_dict, orient="index")
sector_industry_df.reset_index(inplace=True)
sector_industry_df.columns = ["Ticker", "Sector", "Industry"]

# Merge with stock prices dataset
stock_prices = stock_prices.merge(sector_industry_df, on="Ticker", how="left")


**Data Cleaning & Formatting for SQL**

Before inserting the dataset into PostgreSQL, we ensure it is **properly formatted and cleaned**.

**Steps:**
1. **Convert the Date Column**  
   - If the `"Date"` column exists, it is converted into `datetime64` format for SQL compatibility.

2. **Ensure Numerical Columns are Floats**  
   - Selects all numeric columns and converts them into `float`, rounding to **4 decimal places** for precision.

3. **Handle Missing Values (`NaN`)**  
   - Replaces missing values with `NULL` to be properly stored in PostgreSQL.

4. **Rename Columns for SQL Compatibility**  
   - Spaces are replaced with **underscores (`_`)**.  
   - Column names are **lowercased** to avoid case-sensitivity issues.

5. **Final Verification**  
   - Prints the **data types** to confirm everything is correctly formatted before SQL insertion.


In [18]:
# Convert the Date column to datetime format (if it exists)
if "Date" in stock_prices.columns:
    stock_prices["Date"] = pd.to_datetime(stock_prices["Date"])

# Convert all numerical columns to float and round to 4 decimal places
numerical_cols = stock_prices.select_dtypes(include=['number']).columns
stock_prices[numerical_cols] = stock_prices[numerical_cols].astype(float).round(4)

# Replace NaN values with np.nan (which will be converted to NULL in SQL)
stock_prices.fillna(value=np.nan, inplace=True)

# Rename columns to be SQL-friendly (no spaces, lowercase)
stock_prices.columns = stock_prices.columns.str.replace(" ", "_").str.lower()

# Verify after cleaning
print("\n Data is ready for SQL:")
print(stock_prices.dtypes)


 Data is ready for SQL:
date                 datetime64[ns]
ticker                       object
close                       float64
high                        float64
low                         float64
open                        float64
volume                      float64
daily_return                float64
cumulative_return           float64
sma_50                      float64
sma_200                     float64
sharpe_ratio                float64
sector                       object
industry                     object
dtype: object


#### LOAD
The final processed dataset, which includes **stock prices, financial metrics, sector, and industry data**, is exported to a CSV file for further analysis.

**Key Features of the Exported File:**
- **File Name:** `"stock_data_with_sectors.csv"`
- **Data Included:**
  - Stock price data (`Open`, `Close`, `Volume`, etc.).
  - Computed financial indicators (`Daily Return`, `SMA`, `Sharpe Ratio`, etc.).
  - Sector and industry information for each stock.

A confirmation message is printed once the file is successfully created.


In [19]:
# Export the final dataset to CSV
stock_prices.to_csv("stock_data_with_sectors.csv", index=False)

print("CSV file created: 'stock_data_with_sectors.csv' including Sector and Industry information.")


CSV file created: 'stock_data_with_sectors.csv' including Sector and Industry information.


In [20]:
stock_prices

,date,ticker,close,high,low,open,volume,daily_return,cumulative_return,sma_50,sma_200,sharpe_ratio,sector,industry
0,2010-01-04,AC.PA,21.0700,21.0700,20.5987,20.6312,868104.0,-0.0062,0.9938,20.1172,20.9121,0.0162,Consumer Cyclical,Lodging
1,2010-01-04,ACA.PA,6.1843,6.1963,5.9866,6.0035,6105758.0,-0.0062,0.9938,20.1172,20.9121,0.0163,Financial Services,Banks - Regional
2,2010-01-04,AI.PA,30.8401,30.8401,30.3400,30.3400,1114568.0,-0.0062,0.9938,20.1172,20.9121,0.0330,Basic Materials,Specialty Chemicals
3,2010-01-04,AIR.PA,11.2867,11.3351,11.2141,11.2948,2496013.0,-0.0062,0.9938,20.1172,20.9121,0.0390,Industrials,Aerospace & Defense
4,2010-01-04,BN.PA,27.8601,27.9310,27.5249,27.5410,3685834.0,-0.0062,0.9938,20.1172,20.9121,0.0173,Consumer Defensive,Packaged Foods
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153675,2024-12-31,TEP.PA,83.1200,83.9000,81.8800,81.8800,92904.0,0.0171,4.5523,90.0856,96.5467,0.0261,Industrials,Specialty Business Services
153676,2024-12-31,TTE.PA,52.5800,52.5800,51.7918,51.8411,1503880.0,0.0146,2.7080,54.9660,59.9934,0.0193,Energy,Oil & Gas Integrated
153677,2024-12-31,URW.PA,72.7200,73.1400,71.8200,71.8200,129597.0,0.0055,1.4545,74.8536,74.4949,0.0547,Real Estate,REIT - Retail
153678,2024-12-31,VIE.PA,27.1100,27.2100,26.7700,26.8000,764730.0,0.0101,2.3957,28.1834,28.8469,0.0173,Industrials,Waste Management
